In [11]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statistics
import sklearn.metrics as metrics
import tensorflow as tf
from sklearn.metrics import f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import fbeta_score
from sklearn.feature_selection import RFECV
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_csv("../new_train.csv")
test = pd.read_csv("../new_test.csv")

In [3]:
train

,Unnamed: 0,bidder_id,payment_account,address,outcome,auction,merchandise,device,time,country,...,max_url_per_auction,min_url_per_auction,std_url_per_auction,total_no_of_participated_auctions,no_of_auction_exceeds_threshold,percentage_of_auctions_above_threshold,total_no_of_bidded_category,no_of_merchandise_exceeds_threshold,percentage_of_merchandise_above_threshold,on_url_that_has_a_bot_mean
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,18.0,1.0,14.0,24.0,6.0,...,1.0,1.0,0.000000,18.0,0.0,0.000000,1.0,0.0,0.0,1.000000
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,1.0,1.0,2.0,3.0,1.0,...,2.0,2.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.500000
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,1.0,2.0,4.0,1.0,...,1.0,1.0,0.000000,4.0,0.0,0.000000,1.0,0.0,0.0,0.500000
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,1.000000
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,23.0,1.0,53.0,155.0,2.0,...,21.0,1.0,5.644263,23.0,1.0,0.043478,1.0,0.0,0.0,0.010989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,2008,369515b3af4f8ca582f90271d30b14b6r52aw,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,25.0,1.0,4.0,33.0,4.0,...,1.0,1.0,0.000000,25.0,1.0,0.040000,1.0,0.0,0.0,0.500000
2009,2009,f939c17ffc7c39ac9b35b69e5e75179fv9pe2,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000
2010,2010,c806dbb2decba0ed3c4ff5e2e60a74c2wjvbl,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,1.0,1.0,2.0,2.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000
2011,2011,0381a69b7a061e9ace2798fd48f1f537mgq57,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000


In [4]:
train.drop(train.filter(regex="Unname"),axis=1, inplace=True)
test.drop(test.filter(regex="Unname"),axis=1, inplace=True)

In [5]:
X = train.drop(columns=['bidder_id', 'payment_account', 'address', 'outcome','merchandise']) 
y = train['outcome']
X_test = test.drop(columns=['bidder_id', 'payment_account', 'address', 'merchandise'])

### Scaling the training and test dataset

In [6]:
# X.columns

In [6]:
scaler = StandardScaler()
# scaled_features = data.copy()
col_names = ['auction', 'device', 'time', 'country', 'ip', 'url', 'num_bids',
       'num_first_bids', 'num_last_bids', 'time_to_bid', 'inst_resp',
       'perc_inst_resp', 'num_bids_per_auction',
       'num_bids_per_device', 'num_bids_per_country', 'num_bids_per_ip',
       'on_ip_that_has_a_bot_mean', 'ip_entropy', 'url_entropy',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_auction',
       'mean_ip_per_auction', 'max_ip_per_auction', 'min_ip_per_auction',
       'std_ip_per_auction', 'mean_url_per_auction', 'max_url_per_auction',
       'min_url_per_auction', 'std_url_per_auction',
       'total_no_of_participated_auctions', 'no_of_auction_exceeds_threshold',
       'percentage_of_auctions_above_threshold', 'total_no_of_bidded_category',
       'no_of_merchandise_exceeds_threshold',
       'percentage_of_merchandise_above_threshold',
       'on_url_that_has_a_bot_mean']
features = X[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
X[col_names] = features

In [7]:

features = X_test[col_names]
scaler_test = StandardScaler().fit(features.values)
features = scaler_test.transform(features.values)
X_test[col_names] = features

In [19]:
def process_results(arr):
    train_accuracy = []
    train_auc_roc_score = [] 
    train_fbeta = []
    test_accuracy = []
    test_auc_roc_score = []
    test_fbeta = []
    
    for item in arr:
        train_accuracy.append(item[0])
        train_auc_roc_score.append(item[1])
        train_fbeta.append(item[2])
        test_accuracy.append(item[3])
        test_auc_roc_score.append(item[4])
        test_fbeta.append(item[5])
    
    mean_accuracy = np.array(train_accuracy).mean()
    mean_train_auc_roc_score = np.array(train_auc_roc_score).mean()
    mean_train_fbeta = np.array(train_fbeta).mean()
    mean_test_accuracy = np.array(test_accuracy).mean()
    mean_test_auc_roc_score = np.array(test_auc_roc_score).mean()
    mean_test_fbeta = np.array(test_fbeta).mean()
    
    print("final train accuracy: " + str(mean_accuracy))
    print("final train AUC: " + str(mean_train_auc_roc_score))
    print("final train fbeta: " + str(mean_train_fbeta))
    print("final test accuracy: " + str(mean_test_accuracy))
    print("final test AUC: " + str(mean_test_auc_roc_score))
    print("final test fbeta: " + str(mean_test_fbeta))

### model creation


In [8]:
def model_accuracy(selected_features,X,X_test,y):
    
    X = X[selected_features]
    X_test = X_test[selected_features]

    def evaluate(model, X_train, y_train, X_test, y_test):
        print("TRAIN")
        train_predictions = model.predict_proba(X_train)
        train_pred = model.predict(X_train)

        train_accuracy = accuracy_score(y_train, train_pred)
        train_auc_roc_score = roc_auc_score(y_train,train_predictions[:,1])

        print("Classification report")
        print(classification_report(y_train, train_pred, digits = 4))

        print("FBeta Score")
        print(fbeta_score(y_train, train_pred, average='binary', beta=2.0))

        print('Model Performance')
        print('Accuracy = {:0.4f}%.'.format(train_accuracy))
        print('AUC ROC = {:0.4f}%.'.format(train_auc_roc_score))
        print("*" * 100)

        print("TEST")
        test_predictions = model.predict_proba(X_test)
        test_pred = model.predict(X_test)

        test_accuracy = accuracy_score(y_test, test_pred)
        test_auc_roc_score = roc_auc_score(y_test,test_predictions[:,1])

        print("Classification report")
        print(classification_report(y_test, test_pred, digits = 4))

        print("FBeta Score")
        print(fbeta_score(y_test, test_pred, average='binary', beta=2.0))

        print('Model Performance')
        print('Accuracy = {:0.4f}%.'.format(test_accuracy))
        print('AUC ROC = {:0.4f}%.'.format(test_auc_roc_score))
        print("*" * 100)
   
    def create_model(learn_rate=0.01, momentum=0):
        # create model
        model = Sequential()
        model.add(Dense(64, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model
    
    skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.1, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.5, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    
    print("BASED MODEL")
    ann = KerasClassifier(build_fn=create_model, verbose=0)
    ann.fit(X_train, y_train)
    ann_accuracy = evaluate(ann, X_train, y_train, X_test, y_test)

    print("RANDOM SEARCH ANN EXPERIMENT")
    learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
    momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
    batch_size = [10, 20, 40, 60, 80, 100]
    epochs = [10, 50, 100]

    ann_random_grid = dict(learn_rate=learn_rate,momentum=momentum,batch_size=batch_size, epochs=epochs)
    ann = KerasClassifier(build_fn=create_model, verbose=0)
    ann_random = RandomizedSearchCV(estimator = ann, param_distributions = ann_random_grid, n_iter = 100, cv = 2, 
                                verbose=2, random_state=42, n_jobs = -1, scoring='roc_auc')
    ann_random.fit(X_train, y_train)
    ann_best_random = ann_random.best_estimator_
    rand_best_params = ann_random.best_params_
    print(ann_random.best_params_)
    ann_random_accuracy = evaluate(ann_best_random, X_train, y_train, X_test, y_test)


    print("GRID SEARCH ANN EXPERIMENT")
    #     from random search
    #     'momentum': 0.0, 'learn_rate': 0.3, 'epochs': 100, 'batch_size': 10
    # learn_rate = [0.3, 0.1, 0.2]
    # momentum = [0.0, 0.2, 0.4]
    # batch_size = [5, 10, 20]
    # epochs = [50, 100, 150]
    learn_rate = [rand_best_params['learn_rate'],rand_best_params['learn_rate']*1.5,rand_best_params['learn_rate']/1.5]
    momentum = [rand_best_params['momentum'],rand_best_params['momentum']*1.5,rand_best_params['momentum']/1.5]
    batch_size = [rand_best_params['batch_size'],rand_best_params['batch_size']*1.5,rand_best_params['batch_size']/1.5]
    epochs = [rand_best_params['epochs'],rand_best_params['epochs']*1.5,rand_best_params['epochs']/1.5]
    ann_grid = dict(learn_rate=learn_rate,momentum=momentum,batch_size=batch_size, epochs=epochs)
    # ann_grid = rand_best_params
    ann = KerasClassifier(build_fn=create_model, verbose=0)
    ann_grid = GridSearchCV(estimator = ann, param_grid = ann_grid, n_jobs=-1, cv=2, scoring='roc_auc')
    ann_grid.fit(X_train, y_train)
    ann_best_grid = ann_grid.best_estimator_
    grid_best_parms = ann_grid.best_params_
    ann_grid_accuracy = evaluate(ann_best_grid, X_train, y_train, X_test, y_test)


# Other number of Features

In [9]:
selected_features_10 = ['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile']
selected_features_15 = ['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry']
selected_features_20 =['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry', 'num_bids_per_auction', 'max_ip_per_auction', 'inst_resp', 'computers', 'url_entropy']
selected_features_25= ['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry', 'num_bids_per_auction', 'max_ip_per_auction', 'inst_resp', 'computers', 'url_entropy', 'on_ip_that_has_a_bot_mean', 'device', 'home goods', 'time', 'std_url_per_auction']
selected_features_30=['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry', 'num_bids_per_auction', 'max_ip_per_auction', 'inst_resp', 'computers', 'url_entropy', 'on_ip_that_has_a_bot_mean', 'device', 'home goods', 'time', 'std_url_per_auction', 'country', 'sporting goods', 'num_bids', 'perc_inst_resp', 'max_country_per_auction']
selected_features_35=['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry', 'num_bids_per_auction', 'max_ip_per_auction', 'inst_resp', 'computers', 'url_entropy', 'on_ip_that_has_a_bot_mean', 'device', 'home goods', 'time', 'std_url_per_auction', 'country', 'sporting goods', 'num_bids', 'perc_inst_resp', 'max_country_per_auction', 'mean_ip_per_auction', 'std_devices_per_auction', 'max_url_per_auction', 'std_ip_per_auction', 'mean_devices_per_auction']
selected_features_40=['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry', 'num_bids_per_auction', 'max_ip_per_auction', 'inst_resp', 'computers', 'url_entropy', 'on_ip_that_has_a_bot_mean', 'device', 'home goods', 'time', 'std_url_per_auction', 'country', 'sporting goods', 'num_bids', 'perc_inst_resp', 'max_country_per_auction', 'mean_ip_per_auction', 'std_devices_per_auction', 'max_url_per_auction', 'std_ip_per_auction', 'mean_devices_per_auction', 'mean_url_per_auction', 'total_no_of_participated_auctions', 'std_country_per_auction', 'ip_entropy', 'mean_country_per_auction']



In [12]:
model_accuracy(selected_features_10,X,X_test,y)

BASED MODEL


2021-11-06 13:34:05.386747: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-06 13:34:05.441001: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.8978    0.6278    0.7389       266
         1.0     0.5352    0.8571    0.6590       133

    accuracy                         0.7043       399
   macro avg     0.7165    0.7425    0.6989       399
weighted avg     0.7770    0.7043    0.7123       399

FBeta Score
0.7651006711409396
Model Performance
Accuracy = 0.7043%.
AUC ROC = 0.8003%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9890    0.6239    0.7651       577
         1.0     0.0958    0.8519    0.1723        27

    accuracy                         0.6341       604
   macro avg     0.5424    0.7379    0.4687       604
weighted avg     0.9491    0.6341    0.7386       604

FBeta Score
0.33045977011494254
Model Performance
Accuracy = 0.6341%.
AUC ROC = 0.8005%.
*************

2021-11-06 13:34:14.200463: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-06 13:34:14.200463: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-06 13:34:14.200494: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

...

{'momentum': 0.2, 'learn_rate': 0.2, 'epochs': 50, 'batch_size': 40}
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9447    0.8985    0.9210       266
         1.0     0.8151    0.8947    0.8530       133

    accuracy                         0.8972       399
   macro avg     0.8799    0.8966    0.8870       399
weighted avg     0.9015    0.8972    0.8984       399

FBeta Score
0.8775811209439529
Model Performance
Accuracy = 0.8972%.
AUC ROC = 0.9441%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9918    0.8371    0.9079       577
         1.0     0.1966    0.8519    0.3194        27

    accuracy                         0.8377       604
   macro avg     0.5942    0.8445    0.6137       604
weighted avg     0.9562    0.8377    0.8816       604

FBeta Score
0.5111111111111111
Mo

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9395    0.8759    0.9066       266
         1.0     0.7815    0.8872    0.8310       133

    accuracy                         0.8797       399
   macro avg     0.8605    0.8816    0.8688       399
weighted avg     0.8868    0.8797    0.8814       399

FBeta Score
0.8638360175695461
Model Performance
Accuracy = 0.8797%.
AUC ROC = 0.9235%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9891    0.7834    0.8743       577
         1.0     0.1497    0.8148    0.2529        27

    accuracy                         0.7848       604
   macro avg     0.5694    0.7991    0.5636       604
weighted avg     0.9515    0.7848    0.8465       604

FBeta Score
0.4313725490196078
Model Performance
Accuracy = 0.7848%.
AUC ROC = 0.8448%.
**************

In [13]:
model_accuracy(selected_features_15,X,X_test,y)

BASED MODEL
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.8082    0.6654    0.7299       266
         1.0     0.5056    0.6842    0.5815       133

    accuracy                         0.6717       399
   macro avg     0.6569    0.6748    0.6557       399
weighted avg     0.7073    0.6717    0.6804       399

FBeta Score
0.6390449438202248
Model Performance
Accuracy = 0.6717%.
AUC ROC = 0.7182%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9677    0.6222    0.7574       577
         1.0     0.0644    0.5556    0.1154        27

    accuracy                         0.6192       604
   macro avg     0.5160    0.5889    0.4364       604
weighted avg     0.9273    0.6192    0.7287       604

FBeta Score
0.21994134897360704
Model Performance
Accuracy = 0.6192%.
AUC ROC = 0.5947%.
*

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 381, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py", line 567, in roc_auc_score
    return _average_binary_score(
  File "/Users/brennanszeto/Environments/dse

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 381, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py", line 567, in roc_auc_score
    return _average_binary_score(
  File "/Users/brennanszeto/Environments/dse

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

...

{'momentum': 0.2, 'learn_rate': 0.2, 'epochs': 50, 'batch_size': 40}
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9342    0.8534    0.8919       266
         1.0     0.7500    0.8797    0.8097       133

    accuracy                         0.8622       399
   macro avg     0.8421    0.8665    0.8508       399
weighted avg     0.8728    0.8622    0.8645       399

FBeta Score
0.8502906976744187
Model Performance
Accuracy = 0.8622%.
AUC ROC = 0.9162%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9887    0.7591    0.8588       577
         1.0     0.1366    0.8148    0.2340        27

    accuracy                         0.7616       604
   macro avg     0.5627    0.7870    0.5464       604
weighted avg     0.9506    0.7616    0.8309       604

FBeta Score
0.40892193308550184
M

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 381, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py", line 567, in roc_auc_score
    return _average_binary_score(
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_base.py", line 75, in _average_binary_score
    return binary_metric(y_true, y_score, sample_weight=sample_weight)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.8697    0.9286    0.8982       266
         1.0     0.8348    0.7218    0.7742       133

    accuracy                         0.8596       399
   macro avg     0.8523    0.8252    0.8362       399
weighted avg     0.8581    0.8596    0.8569       399

FBeta Score
0.7418856259659967
Model Performance
Accuracy = 0.8596%.
AUC ROC = 0.9236%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9821    0.8544    0.9138       577
         1.0     0.1765    0.6667    0.2791        27

    accuracy                         0.8460       604
   macro avg     0.5793    0.7605    0.5964       604
weighted avg     0.9461    0.8460    0.8854       604

FBeta Score
0.4285714285714286
Model Performance
Accuracy = 0.8460%.
AUC ROC = 0.8323%.
**************

In [14]:
model_accuracy(selected_features_20,X,X_test,y)

BASED MODEL
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.8924    0.5301    0.6651       266
         1.0     0.4813    0.8722    0.6203       133

    accuracy                         0.6441       399
   macro avg     0.6869    0.7011    0.6427       399
weighted avg     0.7554    0.6441    0.6502       399

FBeta Score
0.7503234152652004
Model Performance
Accuracy = 0.6441%.
AUC ROC = 0.7747%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9965    0.4957    0.6620       577
         1.0     0.0820    0.9630    0.1512        27

    accuracy                         0.5166       604
   macro avg     0.5393    0.7293    0.4066       604
weighted avg     0.9556    0.5166    0.6392       604

FBeta Score
0.30588235294117644
Model Performance
Accuracy = 0.5166%.
AUC ROC = 0.7791%.
*

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 381, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py", line 567, in roc_auc_score
    return _average_binary_score(
  File "/Users/brennanszeto/Environments/dse

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 381, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py", line 567, in roc_auc_score
    return _average_binary_score(
  File "/Users/brennanszeto/Environments/dse

...

{'momentum': 0.2, 'learn_rate': 0.2, 'epochs': 50, 'batch_size': 40}
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.8630    0.8759    0.8694       266
         1.0     0.7442    0.7218    0.7328       133

    accuracy                         0.8246       399
   macro avg     0.8036    0.7989    0.8011       399
weighted avg     0.8234    0.8246    0.8239       399

FBeta Score
0.7261724659606656
Model Performance
Accuracy = 0.8246%.
AUC ROC = 0.8619%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9856    0.8302    0.9012       577
         1.0     0.1695    0.7407    0.2759        27

    accuracy                         0.8262       604
   macro avg     0.5775    0.7854    0.5885       604
weighted avg     0.9491    0.8262    0.8733       604

FBeta Score
0.44247787610619466
M

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.8809    0.9173    0.8987       266
         1.0     0.8197    0.7519    0.7843       133

    accuracy                         0.8622       399
   macro avg     0.8503    0.8346    0.8415       399
weighted avg     0.8605    0.8622    0.8606       399

FBeta Score
0.764525993883792
Model Performance
Accuracy = 0.8622%.
AUC ROC = 0.9296%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9818    0.8423    0.9067       577
         1.0     0.1651    0.6667    0.2647        27

    accuracy                         0.8344       604
   macro avg     0.5735    0.7545    0.5857       604
weighted avg     0.9453    0.8344    0.8780       604

FBeta Score
0.4147465437788019
Model Performance
Accuracy = 0.8344%.
AUC ROC = 0.8159%.
***************

In [15]:
model_accuracy(selected_features_25,X,X_test,y)

BASED MODEL
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.7901    0.5376    0.6398       266
         1.0     0.4358    0.7143    0.5413       133

    accuracy                         0.5965       399
   macro avg     0.6129    0.6259    0.5906       399
weighted avg     0.6720    0.5965    0.6070       399

FBeta Score
0.6333333333333333
Model Performance
Accuracy = 0.5965%.
AUC ROC = 0.6999%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9858    0.4801    0.6457       577
         1.0     0.0712    0.8519    0.1314        27

    accuracy                         0.4967       604
   macro avg     0.5285    0.6660    0.3886       604
weighted avg     0.9449    0.4967    0.6227       604

FBeta Score
0.26682134570765664
Model Performance
Accuracy = 0.4967%.
AUC ROC = 0.7233%.
*

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 381, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py", line 567, in roc_auc_score
    return _average_binary_score(
  File "/Users/brennanszeto/Environments/dse

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

...

{'momentum': 0.2, 'learn_rate': 0.2, 'epochs': 50, 'batch_size': 40}
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.8741    0.8872    0.8806       266
         1.0     0.7674    0.7444    0.7557       133

    accuracy                         0.8396       399
   macro avg     0.8208    0.8158    0.8182       399
weighted avg     0.8385    0.8396    0.8390       399

FBeta Score
0.7488653555219364
Model Performance
Accuracy = 0.8396%.
AUC ROC = 0.8840%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9855    0.8250    0.8981       577
         1.0     0.1653    0.7407    0.2703        27

    accuracy                         0.8212       604
   macro avg     0.5754    0.7828    0.5842       604
weighted avg     0.9488    0.8212    0.8700       604

FBeta Score
0.4366812227074236
Mo

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.8058    0.9361    0.8661       266
         1.0     0.8111    0.5489    0.6547       133

    accuracy                         0.8070       399
   macro avg     0.8085    0.7425    0.7604       399
weighted avg     0.8076    0.8070    0.7956       399

FBeta Score
0.5868167202572346
Model Performance
Accuracy = 0.8070%.
AUC ROC = 0.6872%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9759    0.9133    0.9436       577
         1.0     0.2188    0.5185    0.3077        27

    accuracy                         0.8957       604
   macro avg     0.5973    0.7159    0.6256       604
weighted avg     0.9421    0.8957    0.9152       604

FBeta Score
0.4069767441860465
Model Performance
Accuracy = 0.8957%.
AUC ROC = 0.6884%.
**************

In [16]:
model_accuracy(selected_features_30,X,X_test,y)

BASED MODEL
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.7143    0.8083    0.7584       266
         1.0     0.4796    0.3534    0.4069       133

    accuracy                         0.6566       399
   macro avg     0.5969    0.5808    0.5827       399
weighted avg     0.6361    0.6566    0.6412       399

FBeta Score
0.373015873015873
Model Performance
Accuracy = 0.6566%.
AUC ROC = 0.4054%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9549    0.7712    0.8533       577
         1.0     0.0435    0.2222    0.0727        27

    accuracy                         0.7467       604
   macro avg     0.4992    0.4967    0.4630       604
weighted avg     0.9142    0.7467    0.8184       604

FBeta Score
0.12195121951219512
Model Performance
Accuracy = 0.7467%.
AUC ROC = 0.3562%.
**

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

...

{'momentum': 0.2, 'learn_rate': 0.2, 'epochs': 50, 'batch_size': 40}
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.8964    0.8459    0.8704       266
         1.0     0.7230    0.8045    0.7616       133

    accuracy                         0.8321       399
   macro avg     0.8097    0.8252    0.8160       399
weighted avg     0.8386    0.8321    0.8341       399

FBeta Score
0.7867647058823529
Model Performance
Accuracy = 0.8321%.
AUC ROC = 0.7991%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9917    0.8250    0.9007       577
         1.0     0.1855    0.8519    0.3046        27

    accuracy                         0.8262       604
   macro avg     0.5886    0.8384    0.6026       604
weighted avg     0.9556    0.8262    0.8740       604

FBeta Score
0.4956896551724138
Mo

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 381, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py", line 567, in roc_auc_score
    return _average_binary_score(
  File "/Users/brennanszeto/Environments/dse

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.8467    0.9135    0.8788       266
         1.0     0.7946    0.6692    0.7265       133

    accuracy                         0.8321       399
   macro avg     0.8207    0.7914    0.8027       399
weighted avg     0.8293    0.8321    0.8281       399

FBeta Score
0.6909937888198758
Model Performance
Accuracy = 0.8321%.
AUC ROC = 0.9227%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9825    0.8769    0.9267       577
         1.0     0.2022    0.6667    0.3103        27

    accuracy                         0.8675       604
   macro avg     0.5924    0.7718    0.6185       604
weighted avg     0.9476    0.8675    0.8992       604

FBeta Score
0.45685279187817257
Model Performance
Accuracy = 0.8675%.
AUC ROC = 0.8396%.
*************

In [17]:
model_accuracy(selected_features_35,X,X_test,y)

BASED MODEL
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9091    0.3383    0.4932       266
         1.0     0.4133    0.9323    0.5727       133

    accuracy                         0.5363       399
   macro avg     0.6612    0.6353    0.5329       399
weighted avg     0.7438    0.5363    0.5197       399

FBeta Score
0.7451923076923076
Model Performance
Accuracy = 0.5363%.
AUC ROC = 0.8266%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9852    0.3466    0.5128       577
         1.0     0.0599    0.8889    0.1121        27

    accuracy                         0.3709       604
   macro avg     0.5225    0.6178    0.3125       604
weighted avg     0.9439    0.3709    0.4949       604

FBeta Score
0.23575638506876231
Model Performance
Accuracy = 0.3709%.
AUC ROC = 0.7543%.
*

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 381, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py", line 567, in roc_auc_score
    return _average_binary_score(
  File "/Users/brennanszeto/Environments/dse

...

{'momentum': 0.2, 'learn_rate': 0.2, 'epochs': 50, 'batch_size': 40}
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.8300    0.9361    0.8799       266
         1.0     0.8283    0.6165    0.7069       133

    accuracy                         0.8296       399
   macro avg     0.8291    0.7763    0.7934       399
weighted avg     0.8294    0.8296    0.8222       399

FBeta Score
0.6497622820919176
Model Performance
Accuracy = 0.8296%.
AUC ROC = 0.7987%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9887    0.9064    0.9458       577
         1.0     0.2800    0.7778    0.4118        27

    accuracy                         0.9007       604
   macro avg     0.6343    0.8421    0.6788       604
weighted avg     0.9570    0.9007    0.9219       604

FBeta Score
0.5737704918032788
Mo

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 381, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py", line 567, in roc_auc_score
    return _average_binary_score(
  File "/Users/brennanszeto/Environments/dse

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 381, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py", line 567, in roc_auc_score
    return _average_binary_score(
  File "/Users/brennanszeto/Environments/dse

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.8731    0.8797    0.8764       266
         1.0     0.7557    0.7444    0.7500       133

    accuracy                         0.8346       399
   macro avg     0.8144    0.8120    0.8132       399
weighted avg     0.8340    0.8346    0.8343       399

FBeta Score
0.7466063348416291
Model Performance
Accuracy = 0.8346%.
AUC ROC = 0.8247%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9855    0.8267    0.8992       577
         1.0     0.1667    0.7407    0.2721        27

    accuracy                         0.8228       604
   macro avg     0.5761    0.7837    0.5856       604
weighted avg     0.9489    0.8228    0.8711       604

FBeta Score
0.43859649122807015
Model Performance
Accuracy = 0.8228%.
AUC ROC = 0.7954%.
*************

In [18]:
model_accuracy(selected_features_40,X,X_test,y)

BASED MODEL
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.7895    0.6767    0.7287       266
         1.0     0.4971    0.6391    0.5592       133

    accuracy                         0.6642       399
   macro avg     0.6433    0.6579    0.6440       399
weighted avg     0.6920    0.6642    0.6722       399

FBeta Score
0.604551920341394
Model Performance
Accuracy = 0.6642%.
AUC ROC = 0.6661%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9812    0.6343    0.7705       577
         1.0     0.0866    0.7407    0.1550        27

    accuracy                         0.6391       604
   macro avg     0.5339    0.6875    0.4628       604
weighted avg     0.9412    0.6391    0.7430       604

FBeta Score
0.29498525073746307
Model Performance
Accuracy = 0.6391%.
AUC ROC = 0.7326%.
**

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'KerasClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  Fil

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 381, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py", line 567, in roc_auc_score
    return _average_binary_score(
  File "/Users/brennanszeto/Environments/dse

...

[CV] END batch_size=100, epochs=100, learn_rate=0.01, momentum=0.9; total time=   2.3s
[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.9; total time=   5.6s
[CV] END batch_size=80, epochs=50, learn_rate=0.2, momentum=0.4; total time=   2.2s
[CV] END batch_size=100, epochs=50, learn_rate=0.1, momentum=0.4; total time=   2.4s
[CV] END batch_size=10, epochs=50, learn_rate=0.3, momentum=0.2; total time=   3.0s
[CV] END batch_size=100, epochs=10, learn_rate=0.3, momentum=0.9; total time=   1.3s
[CV] END batch_size=60, epochs=100, learn_rate=0.3, momentum=0.6; total time=   2.4s
[CV] END batch_size=40, epochs=100, learn_rate=0.001, momentum=0.9; total time=   2.5s
[CV] END batch_size=80, epochs=50, learn_rate=0.3, momentum=0.0; total time=   1.9s
[CV] END batch_size=100, epochs=50, learn_rate=0.01, momentum=0.4; total time=   2.0s
[CV] END batch_size=100, epochs=10, learn_rate=0.01, momentum=0.2; total time=   1.1s
[CV] END batch_size=80, epochs=10, learn_rate=0.001, momentum=

[CV] END batch_size=100, epochs=100, learn_rate=0.01, momentum=0.9; total time=   2.4s
[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.9; total time=   5.3s
[CV] END batch_size=80, epochs=50, learn_rate=0.2, momentum=0.4; total time=   2.2s
[CV] END batch_size=100, epochs=50, learn_rate=0.1, momentum=0.4; total time=   2.7s
[CV] END batch_size=10, epochs=50, learn_rate=0.3, momentum=0.2; total time=   3.3s
[CV] END batch_size=10, epochs=10, learn_rate=0.001, momentum=0.4; total time=   1.8s
[CV] END batch_size=60, epochs=50, learn_rate=0.01, momentum=0.9; total time=   1.9s
[CV] END batch_size=100, epochs=10, learn_rate=0.1, momentum=0.4; total time=   1.4s
[CV] END batch_size=100, epochs=100, learn_rate=0.1, momentum=0.4; total time=   1.9s
[CV] END batch_size=40, epochs=100, learn_rate=0.3, momentum=0.8; total time=   2.5s
[CV] END batch_size=20, epochs=50, learn_rate=0.2, momentum=0.4; total time=   1.8s
[CV] END batch_size=80, epochs=50, learn_rate=0.01, momentum=0.0

[CV] END batch_size=40, epochs=50, learn_rate=0.2, momentum=0.2; total time=   2.3s
[CV] END batch_size=100, epochs=10, learn_rate=0.2, momentum=0.2; total time=   2.3s
[CV] END batch_size=60, epochs=10, learn_rate=0.3, momentum=0.6; total time=   1.7s
[CV] END batch_size=80, epochs=10, learn_rate=0.2, momentum=0.9; total time=   2.0s
[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.6; total time=   5.2s
[CV] END batch_size=40, epochs=10, learn_rate=0.001, momentum=0.9; total time=   1.7s
[CV] END batch_size=10, epochs=10, learn_rate=0.001, momentum=0.4; total time=   1.7s
[CV] END batch_size=10, epochs=10, learn_rate=0.01, momentum=0.8; total time=   1.9s
[CV] END batch_size=20, epochs=100, learn_rate=0.3, momentum=0.4; total time=   2.9s
[CV] END batch_size=40, epochs=10, learn_rate=0.1, momentum=0.8; total time=   1.4s
[CV] END batch_size=60, epochs=50, learn_rate=0.2, momentum=0.6; total time=   2.3s
[CV] END batch_size=20, epochs=100, learn_rate=0.2, momentum=0.9; to

[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.2; total time=   4.9s
[CV] END batch_size=80, epochs=50, learn_rate=0.3, momentum=0.8; total time=   2.0s
[CV] END batch_size=10, epochs=10, learn_rate=0.01, momentum=0.0; total time=   2.2s
[CV] END batch_size=60, epochs=10, learn_rate=0.001, momentum=0.4; total time=   2.2s
[CV] END batch_size=20, epochs=10, learn_rate=0.01, momentum=0.9; total time=   1.7s
[CV] END batch_size=80, epochs=10, learn_rate=0.1, momentum=0.2; total time=   1.8s
[CV] END batch_size=20, epochs=100, learn_rate=0.1, momentum=0.9; total time=   2.9s
[CV] END batch_size=80, epochs=10, learn_rate=0.01, momentum=0.9; total time=   1.6s
[CV] END batch_size=60, epochs=100, learn_rate=0.2, momentum=0.2; total time=   2.3s
[CV] END batch_size=40, epochs=10, learn_rate=0.1, momentum=0.8; total time=   1.4s
[CV] END batch_size=100, epochs=50, learn_rate=0.01, momentum=0.4; total time=   2.3s
[CV] END batch_size=20, epochs=100, learn_rate=0.2, momentum=0.9; 

[CV] END batch_size=40, epochs=50, learn_rate=0.2, momentum=0.2; total time=   2.3s
[CV] END batch_size=100, epochs=10, learn_rate=0.2, momentum=0.2; total time=   2.1s
[CV] END batch_size=60, epochs=10, learn_rate=0.3, momentum=0.6; total time=   1.6s
[CV] END batch_size=80, epochs=10, learn_rate=0.2, momentum=0.9; total time=   2.0s
[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.6; total time=   5.6s
[CV] END batch_size=40, epochs=10, learn_rate=0.001, momentum=0.9; total time=   1.5s
[CV] END batch_size=100, epochs=10, learn_rate=0.3, momentum=0.9; total time=   1.4s
[CV] END batch_size=60, epochs=100, learn_rate=0.3, momentum=0.6; total time=   2.3s
[CV] END batch_size=40, epochs=100, learn_rate=0.001, momentum=0.9; total time=   2.2s
[CV] END batch_size=80, epochs=50, learn_rate=0.3, momentum=0.0; total time=   1.7s
[CV] END batch_size=60, epochs=50, learn_rate=0.2, momentum=0.6; total time=   2.2s
[CV] END batch_size=100, epochs=10, learn_rate=0.01, momentum=0.2; 